# MTSA


## install


In [1]:
!pip3 install shap



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
!pip3 install mtsa


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
!git clone https://github.com/diegompin/mtsa.git

fatal: destination path 'mtsa' already exists and is not an empty directory.


In [4]:
import os
import shap

import numpy as np

from sklearn.mixture import GaussianMixture
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

from mtsa import files_train_test_split
from mtsa import calculate_aucroc

from mtsa.utils import Wav2Array
from mtsa.features import Array2Mfcc
from mtsa.features import MagnitudeMeanFeatureMfcc
from mtsa.features import MagnitudeStdFeatureMfcc
from mtsa.features import CorrelationFeatureMfcc

from mtsa.models import MFCCMix

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
2024-04-01 15:43:52.859563: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-01 15:43:53.489126: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-04-01 15:43:53.489176: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-04-01 15:43:53.489182: W tensorflow/

## import dados


In [5]:
path_input = "/data/gabriel/valve/id_02"

In [6]:
X_train, X_test, y_train, y_test = files_train_test_split(path_input)

print("Dimensões dos dados de treino: ", X_train.shape)
print("Dimensões dos dados de teste: ",X_test.shape)
print("Dimensões dos rótulos de treino: ",y_train.shape)
print("Dimensões dos rótulos de teste: ",y_test.shape)


Dimensões dos dados de treino:  (588,)
Dimensões dos dados de teste:  (240,)
Dimensões dos rótulos de treino:  (588,)
Dimensões dos rótulos de teste:  (240,)


## Pré-processamento dos sinais:

In [7]:
# Instanciando os objetos
wav2array = Wav2Array(16000)
array2mfcc = Array2Mfcc(16000)
M = MagnitudeMeanFeatureMfcc()
S = MagnitudeStdFeatureMfcc()
C = CorrelationFeatureMfcc()

In [8]:
# Convertendo os sinais para array
array_X_train = wav2array.transform(X_train)
array_X_test = wav2array.transform(X_test)

# Convertendo os arrays para mfcc
mfcc_X_train = array2mfcc.transform(array_X_train)
mfcc_X_test = array2mfcc.transform(array_X_test)

# Convertendo mfcc para as features
M_X_train = M.transform(mfcc_X_train)
S_X_train = S.transform(mfcc_X_train)
C_X_train = C.transform(mfcc_X_train)
M_X_test = M.transform(mfcc_X_test)
S_X_test = S.transform(mfcc_X_test)
C_X_test = C.transform(mfcc_X_test)

# Concatenando as features
concat_X_train = np.concatenate((M_X_train, S_X_train, C_X_train), axis=1)
print("Dimensões das features de treino: ", concat_X_train.shape)

concat_X_test = np.concatenate((M_X_test, S_X_test, C_X_test), axis=1)
print("Dimensões das features de teste: ", concat_X_test.shape)

Dimensões das features de treino:  (588, 230)
Dimensões das features de teste:  (240, 230)


In [9]:
print("Rótulos de treino: ", y_train)
print("Rótulos de teste: ",y_test)

Rótulos de treino:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

## Treinar modelo MFCCMix:

In [12]:
mfccmix = MFCCMix()
mfccmix.fit(X_train, y_train)

print("MFCCMIX")
print("\nTrain: ")
print("Modelo: ", mfccmix.predict(X_train))
print("Real: ", y_train)
print("Score samples: ", mfccmix.score_samples(X_train))

print("\nTest: ")
print("Modelo: ", mfccmix.predict(X_test))
print("Real: ", y_test)
print("Score samples: ", mfccmix.score_samples(X_test))

MFCCMIX

Train: 
Modelo:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [14]:
auc = calculate_aucroc(mfccmix, X_test, y_test)
auc

0.9070833333333334

## Treinar modelo GaussianMixture:

In [15]:
gmm = GaussianMixture()
gmm.fit(concat_X_train, y_train)


print("\n\nGMM")
print("\nTrain: ")
print("Modelo: ", gmm.predict(concat_X_train))
print("Real: ", y_train)
print("Score samples: ", gmm.score_samples(concat_X_train))

print("\nTest: ")
print("Modelo: ", gmm.predict(concat_X_test))
print("Real: ", y_test)
print("Score samples: ", gmm.score_samples(concat_X_test))



GMM

Train: 
Modelo:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 

In [16]:
auc = calculate_aucroc(gmm, concat_X_test, y_test)
auc

0.9084027777777778

## Treinar modelo OneClassSVM:

In [22]:
ocsvm = OneClassSVM(nu=0.05)
ocsvm.fit(concat_X_train, y_train)

print("\n\CVM")
print("\nTrain: ")
print("Modelo: ", ocsvm.predict(concat_X_train))
print("Real: ", y_train)
print("Score samples: ", ocsvm.score_samples(concat_X_train))

print("\nTest: ")
print("Modelo: ", ocsvm.predict(concat_X_test))
print("Real: ", y_test)
print("Score samples: ", ocsvm.score_samples(concat_X_test))


\CVM

Train: 
Modelo:  [ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1 -1 -1  1  1  1  1 -1  1  1  1  1
 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1
  1  1  1  1  1  1  1  1  1 -1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1
 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1 -1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1 -1  1
  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1 -1  1  1  1
  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1 -1  1  1

In [20]:
sub_y_test = y_test
for i in range(len(y_test)):
    if y_test[i] == 0:
        sub_y_test[i] = -1

sub_y_test

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1

In [21]:
auc = calculate_aucroc(ocsvm, concat_X_test, sub_y_test)
auc

0.460625

## Treinar modelo IsolationForest:

In [33]:
IsFo = IsolationForest()
IsFo.fit(concat_X_train, y_train)

print("\n\IsolationForest")
print("\nTrain: ")
print("Modelo: ", IsFo.predict(concat_X_train))
print("Real: ", y_train)
print("Score samples: ", IsFo.score_samples(concat_X_train))

print("\nTest: ")
print("Modelo: ", IsFo.predict(concat_X_test))
print("Real: ", y_test)
print("Score samples: ", IsFo.score_samples(concat_X_test))



\IsolationForest

Train: 
Modelo:  [ 1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1 -1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1
 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1 -1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1 -1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1

In [30]:
sub_y_test = y_test
for i in range(len(y_test)):
    if y_test[i] == 0:
        sub_y_test[i] = -1

sub_y_test

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1

In [32]:
auc = calculate_aucroc(IsFo, concat_X_test, sub_y_test)
auc

0.9300694444444445

## Treinar modelo LocalOutlierFactor:

In [62]:
LOF = LocalOutlierFactor()
LOF.fit(concat_X_train, y_train)

print("\n\LocalOutlierFactor")
print("\nTrain: ")
print("Modelo: ", LOF.fit_predict(concat_X_train))
print("Real: ", y_train)
print("Score samples: ", LOF.negative_outlier_factor_)

print("\nTest: ")
print("Modelo: ", LOF.fit_predict(concat_X_test))
print("Real: ", y_test)
print("Score samples: ", LOF.negative_outlier_factor_)



\LocalOutlierFactor

Train: 
Modelo:  [ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1 -1  1 -1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1 -1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1 -1  1  1 -1 -1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1 -1  1  1  1  1 -1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1
  1 -1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1
  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1

In [63]:
sub_y_test = y_test
for i in range(len(y_test)):
    if y_test[i] == 0:
        sub_y_test[i] = -1

sub_y_test

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1

In [64]:
auc = calculate_aucroc(IsFo, concat_X_test, sub_y_test)
auc

0.8989583333333334

## shape

In [13]:
# Colocar todos os elementos do escore entre 0 e 1 e depois arredondar
def normalizar(x):
  rry = gmm.score_samples(x)

  mn = 1e18
  mx = -1e18
  for c in rry:
    mn = min(mn, c)
    mx = max(mx, c)

  save = []
  cont = 0
  for c in rry:
    if (mx-mn) == 0:
      save.insert(cont, 0)
    else:
      save.insert(cont, (c - mn)/(mx-mn))

    cont += 1

  return np.array(save)

print(normalizar(concat_X_train))

[0.90453997 0.49011583 0.51419209 0.92720933 0.81685636 0.90894755
 0.26215765 0.76701896 0.26749966 0.86857367 0.76911481 0.57403092
 0.91329875 0.92903807 0.89592617 0.88637973 0.91843155 0.92349513
 0.72742493 0.67035192 0.76880819 0.88508725 0.82831915 0.51758101
 0.76402779 0.8612141  0.90340581 0.86821142 0.80942162 0.78504402
 0.88037051 0.81038881 0.61811032 0.8919436  0.73087022 0.83934358
 0.80372958 0.78494876 0.40724917 0.88448172 0.58821372 0.90870949
 0.12504838 0.97084589 1.         0.88676336 0.93107896 0.98011294
 0.59279032 0.94076807 0.970319   0.1885095  0.83639524 0.27649471
 0.13032226 0.8518257  0.87169069 0.71077301 0.81928447 0.60704232
 0.59989561 0.50452515 0.74799729 0.82293279 0.96946087 0.48822556
 0.89268097 0.88846451 0.93425761 0.80177663 0.34532286 0.93177853
 0.30811156 0.8615131  0.87632035 0.78018095 0.92431927 0.8243595
 0.84578509 0.36858265 0.54229052 0.70517863 0.90786802 0.77842389
 0.48022988 0.84236716 0.26060797 0.0792219  0.88691614 0.77941

In [17]:
shap.initjs()

# Explicar as predições no conjunto de teste
explainer = shap.KernelExplainer(normalizar, concat_X_train)
shap_values = explainer.shap_values(concat_X_test)
shap.force_plot(explainer.expected_value, shap_values, concat_X_test)

Using 588 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
  0%|          | 0/240 [00:00<?, ?it/s]

  1%|▏         | 3/240 [00:43<57:47, 14.63s/it]


KeyboardInterrupt: 